## Домашнее задание 4. Определение сарказма с помощью логистической регрессии. Решение

Мы будем использовать датасет из [статьи](https://arxiv.org/abs/1704.05579) "A Large Self-Annotated Corpus for Sarcasm" с более чем 1 млн комментариев с Reddit, размеченных как саркастические или нет. Обработанная версия доступна на Kaggle в виде [Kaggle Dataset](https://www.kaggle.com/danofer/sarcasm).

In [ ]:
PATH_TO_DATA = '../data/train-balanced-sarcasm-part{}.csv'

In [ ]:
# необходимые импорты
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
train_df = pd.concat([pd.read_csv(PATH_TO_DATA.format(i)) for i in range(1, 4)],
                     ignore_index=True)

In [ ]:
train_df.head()

In [ ]:
train_df.info()

Некоторые комментарии отсутствуют, поэтому удалим соответствующие строки.

In [ ]:
train_df.dropna(subset=['comment'], inplace=True)

Заметим, что датасет действительно сбалансирован.

In [ ]:
train_df['label'].value_counts()

Разобьём данные на обучающую и валидационную выборки.

In [ ]:
train_texts, valid_texts, y_train, y_valid = \
        train_test_split(train_df['comment'], train_df['label'], random_state=17)

## Задания:
1. Проанализируйте датасет, постройте графики. Этот [Kernel](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-qiqc) может послужить примером
2. Постройте пайплайн TF-IDF + логистическая регрессия для предсказания сарказма (*label*) по тексту комментария на Reddit (*comment*)
3. Визуализируйте слова/биграммы, наиболее характерные для сарказма (можно использовать [eli5](https://github.com/TeamHG-Memex/eli5))
4. (опционально) Добавьте сабреддиты как новые признаки для улучшения модели. Используйте подход Bag of Words, т.е. рассматривайте каждый сабреддит как отдельный признак.

### Часть 1. Разведочный анализ данных

Распределение длин саркастических и обычных комментариев практически одинаковое.

In [ ]:
train_df.loc[train_df['label'] == 1, 'comment'].str.len().apply(np.log1p).hist(label='sarcastic', alpha=.5)
train_df.loc[train_df['label'] == 0, 'comment'].str.len().apply(np.log1p).hist(label='normal', alpha=.5)
plt.legend();

In [ ]:
from wordcloud import WordCloud, STOPWORDS

In [ ]:
wordcloud = WordCloud(background_color='black', stopwords = STOPWORDS,
                max_words = 200, max_font_size = 100, 
                random_state = 17, width=800, height=400)

Облака слов красивые, но не очень информативные.

In [ ]:
plt.figure(figsize=(16, 12))
wordcloud.generate(str(train_df.loc[train_df['label'] == 1, 'comment']))
plt.imshow(wordcloud);

In [ ]:
plt.figure(figsize=(16, 12))
wordcloud.generate(str(train_df.loc[train_df['label'] == 0, 'comment']))
plt.imshow(wordcloud);

Проанализируем, какие сабреддиты в среднем более «саркастические».

In [ ]:
sub_df = train_df.groupby('subreddit')['label'].agg([np.size, np.mean, np.sum])
sub_df.sort_values(by='sum', ascending=False).head(10)

In [ ]:
sub_df[sub_df['size'] > 1000].sort_values(by='mean', ascending=False).head(10)

Аналогичный анализ по авторам не даёт особых инсайтов. Заметно лишь, что комментарии некоторых авторов были семплированы — видно одинаковое количество саркастических и несаркастических комментариев.

In [ ]:
sub_df = train_df.groupby('author')['label'].agg([np.size, np.mean, np.sum])
sub_df[sub_df['size'] > 300].sort_values(by='mean', ascending=False).head(10)

In [ ]:
sub_df = train_df[train_df['score'] >= 0].groupby('score')['label'].agg([np.size, np.mean, np.sum])
sub_df[sub_df['size'] > 300].sort_values(by='mean', ascending=False).head(10)

In [ ]:
sub_df = train_df[train_df['score'] < 0].groupby('score')['label'].agg([np.size, np.mean, np.sum])
sub_df[sub_df['size'] > 300].sort_values(by='mean', ascending=False).head(10)

### Часть 2. Обучение модели

In [ ]:
# строим биграммы, ограничиваем число признаков
# и минимальную частоту слов
tf_idf = TfidfVectorizer(ngram_range=(1, 2), max_features=50000, min_df=2)
# логистическая регрессия
logit = LogisticRegression(C=1, n_jobs=4, solver='lbfgs', 
                           random_state=17, verbose=1)
# пайплайн sklearn
tfidf_logit_pipeline = Pipeline([('tf_idf', tf_idf), 
                                 ('logit', logit)])

In [ ]:
%%time
tfidf_logit_pipeline.fit(train_texts, y_train)

In [ ]:
%%time
valid_pred = tfidf_logit_pipeline.predict(valid_texts)

In [ ]:
accuracy_score(y_valid, valid_pred)

### Часть 3. Интерпретация модели

In [ ]:
def plot_confusion_matrix(actual, predicted, classes,
                          normalize=False,
                          title='Confusion matrix', figsize=(7,7),
                          cmap=plt.cm.Blues, path_to_save_fig=None):
    """
    Функция для отрисовки матрицы ошибок.
    Нормализацию можно включить, установив `normalize=True`.
    """
    import itertools
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(actual, predicted).T
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Predicted label')
    plt.xlabel('True label')
    
    if path_to_save_fig:
        plt.savefig(path_to_save_fig, dpi=300, bbox_inches='tight')

Матрица ошибок достаточно сбалансирована.

In [ ]:
plot_confusion_matrix(y_valid, valid_pred, 
                      tfidf_logit_pipeline.named_steps['logit'].classes_, figsize=(8, 8))

Действительно, можно распознать некоторые фразы, характерные для сарказма. Например, "yes sure".

In [ ]:
import eli5
eli5.show_weights(estimator=tfidf_logit_pipeline.named_steps['logit'],
                  vec=tfidf_logit_pipeline.named_steps['tf_idf'])

Так что определение сарказма — это просто.
<img src="https://habrastorage.org/webt/1f/0d/ta/1f0dtavsd14ncf17gbsy1cvoga4.jpeg" />

### Часть 4. Улучшение модели

In [ ]:
subreddits = train_df['subreddit']
train_subreddits, valid_subreddits = train_test_split(subreddits, random_state=17)

Используем отдельные TF-IDF векторизаторы для комментариев и сабреддитов. Можно использовать и единый пайплайн, но тогда решение становится менее наглядным. [Пример](https://stackoverflow.com/questions/36731813/computing-separate-tfidf-scores-for-two-different-columns-using-sklearn)

In [ ]:
tf_idf_texts = TfidfVectorizer(ngram_range=(1, 2), max_features=50000, min_df=2)
tf_idf_subreddits = TfidfVectorizer(ngram_range=(1, 1))

Выполним преобразования отдельно для комментариев и сабреддитов.

In [ ]:
%%time
X_train_texts = tf_idf_texts.fit_transform(train_texts)
X_valid_texts = tf_idf_texts.transform(valid_texts)

In [ ]:
X_train_texts.shape, X_valid_texts.shape

In [ ]:
%%time
X_train_subreddits = tf_idf_subreddits.fit_transform(train_subreddits)
X_valid_subreddits = tf_idf_subreddits.transform(valid_subreddits)

In [ ]:
X_train_subreddits.shape, X_valid_subreddits.shape

Затем объединим все признаки вместе.

In [ ]:
from scipy.sparse import hstack
X_train = hstack([X_train_texts, X_train_subreddits])
X_valid = hstack([X_valid_texts, X_valid_subreddits])

In [ ]:
X_train.shape, X_valid.shape

Обучим ту же логистическую регрессию.

In [ ]:
logit.fit(X_train, y_train)

In [ ]:
%%time
valid_pred = logit.predict(X_valid)

In [ ]:
accuracy_score(y_valid, valid_pred)

Как видим, accuracy немного выросла.

## Полезные ссылки:
  - Библиотека машинного обучения [Scikit-learn](https://scikit-learn.org/stable/index.html) (a.k.a. sklearn)
  - Ноутбуки по [логистической регрессии](https://www.kaggle.com/kashnitsky/topic-4-linear-models-part-2-classification) и её применению для [классификации текстов](https://www.kaggle.com/kashnitsky/topic-4-linear-models-part-4-more-of-logit), а также [ноутбук](https://www.kaggle.com/kashnitsky/topic-6-feature-engineering-and-feature-selection) по конструированию и отбору признаков
  - [Kaggle Kernel](https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle) "Approaching (Almost) Any NLP Problem on Kaggle"
  - [ELI5](https://github.com/TeamHG-Memex/eli5) для интерпретации предсказаний модели